<a href="https://colab.research.google.com/github/asadullahhakim/DataScienceProject/blob/main/credit_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Background
contect, case dan dataset dari dokumen ini mengacu ke notes kaggle berikut https://www.kaggle.com/mlg-ulb/creditcardfraud

output dari case ini :
> didapat nya model prediksi/deteksi fraud dan diketahui metric performa nya

#1. importing dependencies


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
#ambil dataset nya

#source of the data set : https://www.kaggle.com/mlg-ulb/creditcardfraud

df0 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/credit card fraud detection siddhardhan/creditcard.csv')

#2. Knowing the context and explore the data set

dari context diketahui deskripsi berikut :

adalah data transaksi credit card dari bank X di eropa, yang terjadi dalam 2 hari
- Features V1, V2, … V28 = the principal components obtainel with PCA,
- Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. unit = second
- feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. unit = USD
- Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

unit per row = transaction done

In [4]:
#Explore data head and tail
df0

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
#explore the basic info about the dataset
df0.info()

#we found that : 
#1. number of column is 31, while number of row is 284807
#2. there are no missing value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [8]:
df0.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [10]:
#explore the distribution between legit transaction and fraud transaction
df0['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [13]:
492/284807*100

0.1727485630620034

dari eksplorasi distribusi transaksi legit vs fraud diatas, terlihat bahwa data distribusi nilai transaksi tersebut sangat unbalanced. ini dikhawatirkan akan mengurangi akurasi dari model prediksi yang akan dibuat 

--> we need to make the dataset to create the model is the balanced dataset

In [16]:
df1_legit_transaction = df0[df0.Class == 0]
df1_fraudulent_transaction = df0[df0.Class == 1]

In [25]:
print(df1_legit_transaction.shape)
print(df1_fraudulent_transaction.shape)

(284315, 31)
(492, 31)


In [30]:
#compare the describptive statistics between the legit vs fraud
df1_legit_transaction.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [31]:
df1_fraudulent_transaction.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

dari sini kita temukan bahwa mean untuk transaksi legit lebih kencil sekitar 2/3 dari transaksi fraud, pun terliha perbedaan yg cukup signifikan di percentil 50 dan 70

In [32]:
#membandingkan rata rata legit vs fraud dari tiap feature
df0.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


dari situ kita lihat lagi perbedaan cukup signifikan dari rata rata value result perhitungan PCA antara data fraud vs data legit.
ini akan jadi core dalam pembuatan prediksi

#3 mempersiapkan data training

kita akan membuat data training yang berimbang, dengan jumlah data fraud = jumlah data legit. (492 data)

In [33]:
#jumlah data fraud
df1_fraudulent_transaction.shape

(492, 31)

In [40]:
#generating 492 data transaksi legit secara random dengan undersampling
df2_legit_sample = df1_legit_transaction.sample(n=492)
print(df2_legit_sample)

            Time        V1        V2  ...       V28  Amount  Class
43763    41644.0  1.301756 -0.040845  ...  0.000763    0.69      0
68982    53215.0 -0.930855  0.618318  ...  0.090140   38.50      0
93445    64423.0 -2.085390 -2.365724  ... -0.041704   55.00      0
99126    66975.0  1.217849 -0.776929  ...  0.019892  104.80      0
71287    54240.0 -3.847114 -3.987326  ... -0.357862  638.00      0
...          ...       ...       ...  ...       ...     ...    ...
25116    33520.0  1.092373 -0.188032  ...  0.023696   29.99      0
223683  143479.0  1.810798 -0.023854  ...  0.021220   63.00      0
44232    41855.0  1.245023 -0.603634  ...  0.007145   60.75      0
139379   83125.0 -1.469751  1.087475  ...  0.013234   32.89      0
189976  128635.0 -2.261377 -1.674289  ...  0.078317  300.00      0

[492 rows x 31 columns]


In [51]:
#mengenerate data set training dengan concatenating 492 data fraud dan 492 random data legit tadi
df3= pd.concat([df2_legit_sample,df1_fraudulent_transaction], axis = 0)
print(df3)

            Time        V1        V2  ...       V28  Amount  Class
43763    41644.0  1.301756 -0.040845  ...  0.000763    0.69      0
68982    53215.0 -0.930855  0.618318  ...  0.090140   38.50      0
93445    64423.0 -2.085390 -2.365724  ... -0.041704   55.00      0
99126    66975.0  1.217849 -0.776929  ...  0.019892  104.80      0
71287    54240.0 -3.847114 -3.987326  ... -0.357862  638.00      0
...          ...       ...       ...  ...       ...     ...    ...
279863  169142.0 -1.927883  1.125653  ...  0.147968  390.00      1
280143  169347.0  1.378559  1.289381  ...  0.186637    0.76      1
280149  169351.0 -0.676143  1.126366  ...  0.194361   77.89      1
281144  169966.0 -3.113832  0.585864  ... -0.253700  245.00      1
281674  170348.0  1.991976  0.158476  ... -0.015309   42.53      1

[984 rows x 31 columns]


In [52]:
df3['Class'].value_counts()

1    492
0    492
Name: Class, dtype: int64

In [53]:
df3.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,95534.581301,0.052444,-0.095754,-0.009029,-0.023766,0.046484,-0.042471,-0.024317,0.020396,0.035931,0.073079,-0.040498,0.024806,0.004663,0.027591,-0.010043,0.046347,-0.048102,-0.023517,0.059829,-0.013402,0.015668,-0.022068,0.060869,0.016292,-0.007066,0.022971,0.037828,-0.004445,85.066220
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [54]:
#memisahkan data jadi feature dan target
df3_feature = df3.drop(columns='Class',axis=1)
df3_target = df3['Class']

print(df3_feature)
print(df3_target)

            Time        V1        V2  ...       V27       V28  Amount
43763    41644.0  1.301756 -0.040845  ... -0.091448  0.000763    0.69
68982    53215.0 -0.930855  0.618318  ...  0.137482  0.090140   38.50
93445    64423.0 -2.085390 -2.365724  ...  0.406626 -0.041704   55.00
99126    66975.0  1.217849 -0.776929  ... -0.089214  0.019892  104.80
71287    54240.0 -3.847114 -3.987326  ... -0.090390 -0.357862  638.00
...          ...       ...       ...  ...       ...       ...     ...
279863  169142.0 -1.927883  1.125653  ...  0.292680  0.147968  390.00
280143  169347.0  1.378559  1.289381  ...  0.389152  0.186637    0.76
280149  169351.0 -0.676143  1.126366  ...  0.385107  0.194361   77.89
281144  169966.0 -3.113832  0.585864  ...  0.884876 -0.253700  245.00
281674  170348.0  1.991976  0.158476  ...  0.002988 -0.015309   42.53

[984 rows x 30 columns]
43763     0
68982     0
93445     0
99126     0
71287     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Nam

In [56]:
#pisahkan dataset jadi training data dan testing data

df3_feature_train, df3_feature_test,df3_target_train,df3_target_test = train_test_split(df3_feature,df3_target, test_size=0.2,stratify=df3_target,random_state=2)

In [57]:
print(df3_feature.shape,df3_feature_test.shape,df3_feature_train.shape)

(984, 30) (197, 30) (787, 30)


In [59]:
#4 melakukan training terhadap model
#akan menggunakan logistic regression

model = LogisticRegression()

In [61]:
model.fit(df3_feature_train,df3_target_train)

LogisticRegression()

#4. Model Evaluation
mencari performa model / accuracy score

In [70]:
df4_prediction_train = model.predict(df3_feature_train)
train_data_accuracy = accuracy_score(df4_prediction_train,df3_target_train)

In [71]:
print('accuracy dari data training :',train_data_accuracy)

accuracy dari data training : 0.9440914866581956


In [72]:
df4_prediction_test = model.predict(df3_feature_test)
test_data_accuracy = accuracy_score(df4_prediction_test,df3_target_test)

In [74]:
df4_prediction_test

array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0])

In [73]:
print('accuracy dari data test :',test_data_accuracy)

accuracy dari data test : 0.9187817258883249
